# NASA Turbofan Engine Dataset Visualization 3

##### 여기서부터는 엔진별, 센서별로 분할된 데이터를 센서별로 1번부터 100번 엔진까지의 데이터를 한 차트에 보여줄 수 있도록 병합하는 코드를 만들겠다. 

In [52]:
import pandas as pd
import numpy as np
from pathlib import Path
from pathlib import PurePath
from pandas import DataFrame
import xlwings as xw
import xlsxwriter as xl

### NASA Turbofan Engine Dataset Visualization 2에서 지정한 변수들 불러오기
    
#### FD001,2,3,4 Train 데이터셋 구조 파악 (FD001,2,3,4 Test 데이터 불러오기)

### 1번 ~ 100번 엔진까지, 각 센서별 데이터로 재추출하는 코드

이전에는 엔진별로 서로 다른 종류의 모든 센서 데이터를 하나의 시계열 그래프로 나타내었다면,
이번에는 엔진번호(Unit_Number), 비행시간(Time), 센서 1개에서 나온 데이터만 추출해보겠다.
그리고 나중에는 이를 바탕으로 특정 한 센서를 중심으로 각 엔진들을 join한 형태의 테이블을 작성해보겠다.

그전에 우선 FD001의 train에서 4번엔진의 비행 싸이클인 'Time','Fan Inlet Temp' 데이터만 추출해보겠다.
필요한 코드는 아래와 같다.

#### FD001 train set 센서별 데이터 병합

In [90]:
# 01. Fan Inlet Temp


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Fan Inlet Temp 파일 병합이 완료되었습니다.')

FD001 train Fan Inlet Temp 파일 병합이 완료되었습니다.


해당 엑셀파일을 열어보면 아래와 같다.

In [54]:
example = pd.read_excel('C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/FD001 train Fan Inlet Temp.xlsx')

example

,Time (Flight Cycle),1번 엔진,2번 엔진,3번 엔진,4번 엔진,5번 엔진,6번 엔진,7번 엔진,8번 엔진,9번 엔진,...,91번 엔진,92번 엔진,93번 엔진,94번 엔진,95번 엔진,96번 엔진,97번 엔진,98번 엔진,99번 엔진,100번 엔진
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,397,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397,398,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,399,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
# 02. LPC Outlet Temp


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/02. LPC Outlet Temp/'

excel_file = folder + 'FD001 train LPC Outlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/02. LPC Outlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 02. LPC Outlet Temp 파일 병합이 완료되었습니다.')

FD001 train 02. LPC Outlet Temp 파일 병합이 완료되었습니다.


In [80]:
# 03. HPC Outlet Temp


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/03. HPC Outlet Temp/'

excel_file = folder + 'FD001 train HPC Outlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/03. HPC Outlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 03. HPC Outlet Temp 파일 병합이 완료되었습니다.')

FD001 train HPC Outlet Temp 파일 병합이 완료되었습니다.


In [81]:
# 04. LPT Outlet Temp


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/04. LPT Outlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 04. LPT Outlet Temp 파일 병합이 완료되었습니다.')

FD001 train LPT Outlet Temp 파일 병합이 완료되었습니다.


In [79]:
# 05. Fan Inlet Press


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/05. Fan Inlet Press/'

excel_file = folder + 'FD001 train Fan Inlet Press.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')



# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)



# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)



# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/05. Fan Inlet Press/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 05. Fan Inlet Press 파일 병합이 완료되었습니다.')

FD001 train Fan Inlet Press 파일 병합이 완료되었습니다.


In [82]:
# 06. Bypass Duct Press


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Bypass Duct Press 파일 병합이 완료되었습니다.')

FD001 train Bypass Duct Press 파일 병합이 완료되었습니다.


In [83]:
# 07. Total HPC Outlet Press


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Total HPC Outlet Press 파일 병합이 완료되었습니다.')

FD001 train Total HPC Outlet Press 파일 병합이 완료되었습니다.


In [84]:
# 08. Physical Fan Speed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Physical Fan Speed 파일 병합이 완료되었습니다.')

FD001 train Physical Fan Speed 파일 병합이 완료되었습니다.


In [85]:
# 09. Physical Core Speed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Physical Core Speed 파일 병합이 완료되었습니다.')

FD001 train Physical Core Speed 파일 병합이 완료되었습니다.


In [86]:
# 10. Engine Press Ratio


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Engine Press Ratio 파일 병합이 완료되었습니다.')

FD001 train Engine Press Ratio 파일 병합이 완료되었습니다.


In [87]:
# 11. Static HPC Outlet Press


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Static HPC Outlet Press 파일 병합이 완료되었습니다.')

FD001 train Static HPC Outlet Press 파일 병합이 완료되었습니다.


In [65]:
# 12. Fuel Flow Ratio to Ps30


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Fuel Flow Ratio to Ps30 파일 병합이 완료되었습니다.')

완료되었습니다.


In [66]:
# 13. Corrected Fan Speed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Corrected Fan Speed 파일 병합이 완료되었습니다.')

완료되었습니다.


In [67]:
# 14. Corrected Corr Speed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Corrected Corr Speed 파일 병합이 완료되었습니다.')

완료되었습니다.


In [68]:
# 15. Bypass Ratio


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Bypass Ratio 파일 병합이 완료되었습니다.')

완료되었습니다.


In [89]:
# 16. Burner Fuel-Air ratio


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Burner Fuel-Air ratio.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train Burner Fuel-Air ratio 파일 병합이 완료되었습니다.')

FD001 train Burner Fuel-Air ratio 파일 병합이 완료되었습니다.


In [88]:
# 17. Bleed Enthalpy


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('FD001 train 17. Bleed Enthalpy 파일 병합이 완료되었습니다.')

FD001 train 17. Bleed Enthalpy 파일 병합이 완료되었습니다.


In [71]:
# 18. Demanded Fan Speed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('완료되었습니다.')

완료되었습니다.


In [72]:
# 19. Demanded Corrected Fan Speed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('완료되었습니다.')

완료되었습니다.


In [73]:
# 20. HPT Collant Bleed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('완료되었습니다.')

완료되었습니다.


In [74]:
# 21. LPT_Coolant_Bleed


# 1. 엑셀 파일 저장 경로

folder = 'C:/Users/jhj\Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/'

excel_file = folder + 'FD001 train Fan Inlet Temp.xlsx'

workbook = xlsxwriter.Workbook(excel_file)          # 워크북 객체 생성

worksheet = workbook.add_worksheet()                # 워크시트 생성

worksheet.write(0,0,'Time (Flight Cycle)')


# 2. 엔진 번호 입력

for i in range(100):

    Engine_Number = str(i+1) + '번 엔진'

    worksheet.write(0,i+1,Engine_Number)


# 3. x축에 해당하는 'Time (Cycle)'을 입력

for i in range(400):

    worksheet.write(i+1,0,i+1)


# 4. 엔진별, 센서별 분산된 데이터를 불러와서 병합

for j in range(100):

    for i in range(400):
    
        direct = "='C:/Users/jhj/Desktop/NASA_Turbofan_Engine_excel_data/센서별 엔진 데이터 모음/train/FD001/01. Fan Inlet Temp/[" + str(j+1) + "엔진.xlsx]Sheet1'!$C"

        cell = str(i+2)

        data = direct + cell

        worksheet.write(i+1,j+1,data)


workbook.close()

print('완료되었습니다.')

완료되었습니다.
